In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import gc

In [16]:
channelset = set()
for filename in os.listdir("FullData"):
    if os.path.isfile("FullData/" + filename):
        # get channel name
        channelset.add(filename.split("#")[1].split(".")[0])
for channel in channelset:
    print("\"" + channel + "\": {\"Vtuber\": False, \"Male\": True},")

"rdulive": {"Vtuber": False, "Male": True},
"beardageddon": {"Vtuber": False, "Male": True},
"giantwaffle": {"Vtuber": False, "Male": True},
"aicandii": {"Vtuber": False, "Male": True},
"clauvio": {"Vtuber": False, "Male": True},
"enviosity": {"Vtuber": False, "Male": True},
"amnotasadist": {"Vtuber": False, "Male": True},
"bateson87": {"Vtuber": False, "Male": True},
"itsjstn": {"Vtuber": False, "Male": True},
"cirno_tv": {"Vtuber": False, "Male": True},
"robraven": {"Vtuber": False, "Male": True},
"maxylobes": {"Vtuber": False, "Male": True},
"asianguystream": {"Vtuber": False, "Male": True},
"zentreya": {"Vtuber": False, "Male": True},
"iahfy": {"Vtuber": False, "Male": True},
"alicesawyer": {"Vtuber": False, "Male": True},
"singsing": {"Vtuber": False, "Male": True},
"burkeblack": {"Vtuber": False, "Male": True},
"fextralife": {"Vtuber": False, "Male": True},
"benfruit": {"Vtuber": False, "Male": True},
"blinkx_": {"Vtuber": False, "Male": True},
"aavak": {"Vtuber": False, "Male": 

In [ ]:
{
    "rdulive": {"Vtuber": False, "Male": True},
    "beardageddon": {"Vtuber": False, "Male": True},
    "giantwaffle": {"Vtuber": False, "Male": True},
    "aicandii": {"Vtuber": True, "Male": False},
    "clauvio": {"Vtuber": True, "Male": False},
    "enviosity": {"Vtuber": False, "Male": True},
    "amnotasadist": {"Vtuber": True, "Male": True},
    "bateson87": {"Vtuber": False, "Male": True},
    "itsjstn": {"Vtuber": False, "Male": True},
    "cirno_tv": {"Vtuber": True, "Male": True},
    "robraven": {"Vtuber": False, "Male": True},
    "maxylobes": {"Vtuber": False, "Male": True},
    "asianguystream": {"Vtuber": False, "Male": True},
    "zentreya": {"Vtuber": True, "Male": False},
    "iahfy": {"Vtuber": True, "Male": False},
    "alicesawyer": {"Vtuber": True, "Male": False},
    "singsing": {"Vtuber": False, "Male": True},
    "burkeblack": {"Vtuber": False, "Male": True},
    "fextralife": {"Vtuber": False, "Male": True},
    "benfruit": {"Vtuber": False, "Male": True},
    "blinkx_": {"Vtuber": False, "Male": False},
    "aavak": {"Vtuber": False, "Male": True},
    "gothicbunni": {"Vtuber": True, "Male": False},
    "anniefuchsia": {"Vtuber": False, "Male": False},
    "hera": {"Vtuber": False, "Male": True},
    "therunningmanz": {"Vtuber": False, "Male": True},
    "vanessa_lopez_official": {"Vtuber": False, "Male": False},
    "tokki": {"Vtuber": False, "Male": False},
    "preachlfw": {"Vtuber": False, "Male": True},
    "sharonqueen": {"Vtuber": False, "Male": False},
    "ballindani": {"Vtuber": True, "Male": False},
    "javathecuptv": {"Vtuber": True, "Male": True},
    "lewpac": {"Vtuber": False, "Male": True},
    "sunshinevrc": {"Vtuber": True, "Male": False},
    "danehearth": {"Vtuber": False, "Male": True},
    "barbarousking": {"Vtuber": False, "Male": True},
    "tenha": {"Vtuber": False, "Male": True},
    "naguura": {"Vtuber": False, "Male": False},
    "veibae": {"Vtuber": True, "Male": False},
    "dexbonus": {"Vtuber": False, "Male": False},
    "retrogaijin": {"Vtuber": False, "Male": True},
    "faide": {"Vtuber": False, "Male": True},
    "dangheesling": {"Vtuber": False, "Male": True},
    "harukakaribu": {"Vtuber": True, "Male": False},
    "ds_lily": {"Vtuber": False, "Male": False},
    "orzanel": {"Vtuber": False, "Male": True},
    "peachmilky": {"Vtuber": True, "Male": False},
    "kandyland": {"Vtuber": False, "Male": False},
    "teepee": {"Vtuber": False, "Male": True},
    "xqc": {"Vtuber": False, "Male": True},
    "atk": {"Vtuber": False, "Male": True},
    "aiyanya": {"Vtuber": True, "Male": False},
    "faxuty": {"Vtuber": False, "Male": True},
    "pengu": {"Vtuber": False, "Male": True},
    "lyasyaa": {"Vtuber": False, "Male": False},
    "arnie": {"Vtuber": False, "Male": True},
    "fatalseductions": {"Vtuber": False, "Male": False},
    "cr1tdota": {"Vtuber": False, "Male": True},
    "missmikkaa": {"Vtuber": False, "Male": False},
    "thejrm_": {"Vtuber": False, "Male": True},
    "bestdadtuber": {"Vtuber": True, "Male": True},
    "elosanta": {"Vtuber": False, "Male": True},
    "imls": {"Vtuber": False, "Male": True},
    "ariasaki": {"Vtuber": False, "Male": False},
    "starsmitten": {"Vtuber": True, "Male": False},
    "basilwoof": {"Vtuber": True, "Male": True},
    "cerianmusic": {"Vtuber": False, "Male": False},
    "dyanna": {"Vtuber": False, "Male": False},
    "abdulhd": {"Vtuber": False, "Male": True},
    "spear_shot": {"Vtuber": False, "Male": True},
    "flats": {"Vtuber": False, "Male": True},
    "boraslegend": {"Vtuber": False, "Male": True},
    "trickywi": {"Vtuber": True, "Male": False},
    "gingitv": {"Vtuber": False, "Male": True},
    "maggiekarp_": {"Vtuber": False, "Male": False},
    "axel_tv": {"Vtuber": False, "Male": True},
    "72hrs": {"Vtuber": False, "Male": True},
    "mitchjones": {"Vtuber": False, "Male": True},
    "gateoo": {"Vtuber": True, "Male": True},
    "katarinafps": {"Vtuber": False, "Male": False},
    "kurumii": {"Vtuber": True, "Male": False},
    "jokerdtv": {"Vtuber": False, "Male": True},
    "yuzupyon": {"Vtuber": True, "Male": False},
    "bunniejin": {"Vtuber": False, "Male": False},
    "thestockguy": {"Vtuber": False, "Male": True},
    "vombuz": {"Vtuber": False, "Male": True},
    "oroboro": {"Vtuber": False, "Male": True},
    "limealicious": {"Vtuber": True, "Male": False},
    "aleks": {"Vtuber": False, "Male": True},
    "ikumi": {"Vtuber": True, "Male": False},
    "saintvicious": {"Vtuber": False, "Male": True},
    "kragiee": {"Vtuber": False, "Male": True},
    "maven": {"Vtuber": False, "Male": True},
    "meatihs": {"Vtuber": False, "Male": True},
    "alkaizerx": {"Vtuber": False, "Male": True},
    "hambinooo": {"Vtuber": False, "Male": True},
    "robcdee": {"Vtuber": False, "Male": True},
    "emilyywang": {"Vtuber": False, "Male": False},
    "baker": {"Vtuber": False, "Male": True},
    "travpiper": {"Vtuber": True, "Male": False},
    "nalithea": {"Vtuber": False, "Male": True},
    "lhcloudy27-1": {"Vtuber": False, "Male": True},
    "carmilllia": {"Vtuber": True, "Male": False},
    "chrisnxtdoor": {"Vtuber": False, "Male": True},
    "jhinxx_": {"Vtuber": True, "Male": False},
    "ainrun": {"Vtuber": False, "Male": True},
    "greekgodx": {"Vtuber": False, "Male": True},
    "martyguyz": {"Vtuber": False, "Male": True},
    "akiwoo": {"Vtuber": True, "Male": False},
    "gammainkk": {"Vtuber": True, "Male": False},
    "dunkorslam": {"Vtuber": False, "Male": True},
    "liqudwifi": {"Vtuber": False, "Male": True},
    "merk": {"Vtuber": False, "Male": True},
    "yagurlshelly": {"Vtuber": True, "Male": False},
    "antielitz": {"Vtuber": False, "Male": True},
    "shaiquera": {"Vtuber": False, "Male": False},
    "nepenthez": {"Vtuber": False, "Male": True},
    "kxpture": {"Vtuber": False, "Male": True},
    "spyro_za": {"Vtuber": False, "Male": True},
    "thatpunchkid": {"Vtuber": False, "Male": True},
    "leopard": {"Vtuber": False, "Male": True},
    "babynikki": {"Vtuber": False, "Male": False},
    "dailydasher": {"Vtuber": False, "Male": True},
    "klean": {"Vtuber": False, "Male": True},
    "snuffy": {"Vtuber": True, "Male": False},
    "unknownxarmy": {"Vtuber": False, "Male": True},
    "chrisheroes": {"Vtuber": False, "Male": True},
    "rubee": {"Vtuber": True, "Male": False},
    "apricot": {"Vtuber": True, "Male": False},
    "break": {"Vtuber": False, "Male": True},
    "xaeela": {"Vtuber": True, "Male": False},
    "ariannafoxton": {"Vtuber": True, "Male": False},
    "sukidingels": {"Vtuber": True, "Male": False},
    "shuusakurai": {"Vtuber": True, "Male": True},
    "ninaninin": {"Vtuber": True, "Male": False},
    "niamocha": {"Vtuber": True, "Male": False},
    "dyarikku": {"Vtuber": True, "Male": False},
    "thijs": {"Vtuber": False, "Male": True},
    "p4wnyhof": {"Vtuber": False, "Male": True},
    "lilsimsie": {"Vtuber": False, "Male": False},
    "redbeard": {"Vtuber": False, "Male": True},
    "sleepy_project": {"Vtuber": True, "Male": False},
    "husher_x": {"Vtuber": True, "Male": False},
    "deltiasgaming": {"Vtuber": False, "Male": True},
    "39daph": {"Vtuber": True, "Male": False},
    "skill4ltu": {"Vtuber": False, "Male": True},
    "otzdarva": {"Vtuber": False, "Male": True},
    "oliviamonroe": {"Vtuber": True, "Male": False},
    "murdercrumpet": {"Vtuber": True, "Male": False},
    "jingggxd": {"Vtuber": False, "Male": True},
    "prod": {"Vtuber": False, "Male": True},
    "dinossindgeil": {"Vtuber": False, "Male": True},
    "grimmivt": {"Vtuber": True, "Male": False},
    "beast1k": {"Vtuber": False, "Male": True},
    "ksonsouchou": {"Vtuber": True, "Male": False},
    "mary": {"Vtuber": False, "Male": False},
    "girl_dm_": {"Vtuber": True, "Male": False},
    "nyanners": {"Vtuber": True, "Male": False},
    "hiswattson": {"Vtuber": False, "Male": True},
    "shadeless": {"Vtuber": False, "Male": True},
    "ironmouse": {"Vtuber": True, "Male": False},
    "aspiringspike": {"Vtuber": False, "Male": True},
    "lol_nemesis": {"Vtuber": False, "Male": True},
    "zethiann": {"Vtuber": False, "Male": True},
    "baalorlord": {"Vtuber": False, "Male": True},
    "curvyelephant": {"Vtuber": False, "Male": True},
    "pokelawls": {"Vtuber": False, "Male": True},
    "emongg": {"Vtuber": False, "Male": True},
    "dezignful": {"Vtuber": False, "Male": True},
    "kyedae": {"Vtuber": False, "Male": False},
    "incon": {"Vtuber": False, "Male": True},
    "silvervale": {"Vtuber": true, "Male": False},
    "zachmazer": {"Vtuber": False, "Male": True},
    "amaz": {"Vtuber": False, "Male": True},
    "ambiguousamphibian": {"Vtuber": False, "Male": True},
    "a2guapo": {"Vtuber": False, "Male": True},
    "ohmwrecker": {"Vtuber": False, "Male": True},
    "faith": {"Vtuber": False, "Male": False},
    "willerz": {"Vtuber": False, "Male": True},
    "cringer": {"Vtuber": False, "Male": True},
    "salt": {"Vtuber": False, "Male": True},
    "olofmeister": {"Vtuber": False, "Male": True},
    "broyouwack": {"Vtuber": False, "Male": True},
    "stal": {"Vtuber": True, "Male": False},
    "feer": {"Vtuber": False, "Male": True},
    "lhcloudy27": {"Vtuber": False, "Male": True},
    "sevensins": {"Vtuber": False, "Male": True},
    "gmhikaru": {"Vtuber": False, "Male": True},
    "scrapie": {"Vtuber": False, "Male": True},
    "jonbams": {"Vtuber": False, "Male": True},
    "abe": {"Vtuber": True, "Male": True},
    "aceofspadesow": {"Vtuber": False, "Male": True},
    "bogur": {"Vtuber": False, "Male": True},
    "komemos": {"Vtuber": False, "Male": True},
    "spicyuuu": {"Vtuber": False, "Male": False},
    "goldbridgetv": {"Vtuber": False, "Male": True},
    "noodlewolfvt": {"Vtuber": True, "Male": False},
    "cohhcarnage": {"Vtuber": False, "Male": True},
    "hiko": {"Vtuber": False, "Male": True},
    "ac7ionman": {"Vtuber": False, "Male": True},
    "moosebrother": {"Vtuber": False, "Male": True},
    "maudado": {"Vtuber": False, "Male": True},
    "nediavr": {"Vtuber": True, "Male": False},
    "sorry": {"Vtuber": True, "Male": False},
    "sp4zie": {"Vtuber": False, "Male": True},
    "kyokeru": {"Vtuber": True, "Male": False},
    "shrodingerlee": {"Vtuber": False, "Male": True},
    "overezeggs": {"Vtuber": True, "Male": False},
    "rennslyaer": {"Vtuber": True, "Male": False},
    "dannyaarons": {"Vtuber": False, "Male": True},
    "nohandsgamer": {"Vtuber": False, "Male": True},
    "okcode": {"Vtuber": True, "Male": True},
    "hashtag_stokes": {"Vtuber": False, "Male": True},
    "trey24k": {"Vtuber": False, "Male": True},
    "hens333": {"Vtuber": False, "Male": True},
    "kuugels": {"Vtuber": False, "Male": False},
    "admiralbulldog": {"Vtuber": False, "Male": True},
    "naowh": {"Vtuber": False, "Male": True},
    "taxi2g": {"Vtuber": False, "Male": True},
    "peeve": {"Vtuber": False, "Male": True},
    "elajjaz": {"Vtuber": False, "Male": True},
    "longlivequebec": {"Vtuber": False, "Male": True},
    "zealsambitious": {"Vtuber": False, "Male": True},
    "kyundere": {"Vtuber": True, "Male": False},
    "negnasu": {"Vtuber": True, "Male": False},
    "shadder2k": {"Vtuber": False, "Male": True},
    "nugiyen": {"Vtuber": False, "Male": True},
    "projektmelody": {"Vtuber": True, "Male": False},
    "kawaiigrin": {"Vtuber": True, "Male": False},
    "nymn": {"Vtuber": False, "Male": True},
    "zyblol": {"Vtuber": False, "Male": True},
    "gorgc": {"Vtuber": False, "Male": True},
    "circon": {"Vtuber": False, "Male": True},
    "castro_1021": {"Vtuber": False, "Male": True},
    "sinatraa": {"Vtuber": False, "Male": True},
    "tobii": {"Vtuber": False, "Male": True},
    "manvsgame": {"Vtuber": False, "Male": True},
    "florryworry": {"Vtuber": False, "Male": True},
    "2dkiri": {"Vtuber": False, "Male": False},
    "chey": {"Vtuber": True, "Male": False},
    "tobs": {"Vtuber": True, "Male": False},
    "tanetenshi": {"Vtuber": True, "Male": False},
    "midbeast": {"Vtuber": False, "Male": True},
    "onikanavt": {"Vtuber": True, "Male": False},
    "guru": {"Vtuber": False, "Male": True},
    "memorizer92": {"Vtuber": False, "Male": True},
    "meowmoonified": {"Vtuber": True, "Male": False},
    "thecrimsonblur": {"Vtuber": False, "Male": True},
    "jenazad": {"Vtuber": False, "Male": True},
    "diegosaurs": {"Vtuber": False, "Male": True},
    "unrational": {"Vtuber": False, "Male": True},
    "tsm_viss": {"Vtuber": False, "Male": True},
    "ceremor": {"Vtuber": False, "Male": True},
    "hirona": {"Vtuber": False, "Male": False},
    "zetalot": {"Vtuber": False, "Male": True},
    "momo": {"Vtuber": True, "Male": False},
    "fragnance": {"Vtuber": False, "Male": True},
    "iddqd": {"Vtuber": False, "Male": True},
    "munchking": {"Vtuber": False, "Male": True},
    "solorenektononly": {"Vtuber": False, "Male": True},
    "danucd": {"Vtuber": False, "Male": False},
    "hutchmf": {"Vtuber": False, "Male": True},
    "butters": {"Vtuber": False, "Male": True},
    "develique": {"Vtuber": False, "Male": False},
    "auteru": {"Vtuber": True, "Male": False},
    "mande": {"Vtuber": False, "Male": True},
    "tgh_sr": {"Vtuber": False, "Male": True},
    "keshaeuw": {"Vtuber": False, "Male": True},
    "uhsnow": {"Vtuber": False, "Male": True},
    "canniny": {"Vtuber": False, "Male": False},
    "richard_hammer": {"Vtuber": False, "Male": True},
    "foxplushy": {"Vtuber": True, "Male": False},
    "sequisha": {"Vtuber": False, "Male": True},
    "heyzeusherestoast": {"Vtuber": False, "Male": True},
    "tommykaylive": {"Vtuber": False, "Male": True},
    "kyliebitkin": {"Vtuber": False, "Male": False},
    "nidas": {"Vtuber": False, "Male": True},
    "jackcashew": {"Vtuber": False, "Male": True},
    "pohx": {"Vtuber": False, "Male": True},
    "amouranth": {"Vtuber": False, "Male": False},
    "sakuratsubasa": {"Vtuber": True, "Male": False},
    "vividlyvivi": {"Vtuber": True, "Male": False},
    "mmorpg": {"Vtuber": False, "Male": True},
    "tenma": {"Vtuber": True, "Male": False},
    "gingy": {"Vtuber": False, "Male": False},
    "tenz": {"Vtuber": False, "Male": True},
    "omie": {"Vtuber": False, "Male": True},
    "sick_nerd": {"Vtuber": False, "Male": True},
    "ming": {"Vtuber": False, "Male": True},
    "premiertwo": {"Vtuber": False, "Male": True},
    "cookielolxx": {"Vtuber": False, "Male": True},
    "bobbypoffgaming": {"Vtuber": False, "Male": True},
    "sco": {"Vtuber": False, "Male": True},
    "polysypher": {"Vtuber": False, "Male": True},
    "spammiej": {"Vtuber": False, "Male": True},
    "fenohs": {"Vtuber": False, "Male": True},
    "timo_redbeard": {"Vtuber": False, "Male": True},
    "loeya": {"Vtuber": False, "Male": False},
    "birdieboba": {"Vtuber": True, "Male": False},
    "jpf_14-1": {"Vtuber": False, "Male": True},
    "nats": {"Vtuber": False, "Male": True},
    "donkoalachannel": {"Vtuber": True, "Male": True},
    "cerisevt": {"Vtuber": true, "Male": True},
    "jpf_14": {"Vtuber": False, "Male": True},
    "lumituber": {"Vtuber": False, "Male": True},
    "deadlyslob": {"Vtuber": False, "Male": True},
    "marimari_en": {"Vtuber": False, "Male": True}
    }

In [2]:
# TODO: add a field for trusted users. A user is trusted if they're a moderator or a long-time sub.
# TODO: Discard bot messages. Consider also discarding mod messages since they're often the only ones allowed to send links.
train = 0
test = 0
data = {} 
# of the form:
# {"channel1":
#    {
#     "messages": ["messsage1", "message2"...],
#     "bad_messages": [4, 18...], (indices of messages)
#     "viewers": 482 (average)
#    }
#  "channel2":
#   {...}
# }


# look through every data file
for filename in os.listdir("FullData"):
    if os.path.isfile("FullData/" + filename):
        # get channel name
        channel = filename.split("#")[1].split(".")[0]
        if not channel in data:
            data[channel] = {"viewers": [], "messages": [], "bad_messages": []}
        with open("FullData/" + filename, encoding='utf-8') as file:
            lines = []
            for line in file.readlines():
                # only care about timestamped lines. Others are overhead data that we don't mind.
                if line[0] == "[":
                    # get rid of the timestamp, we only want the message itself.
                    line = line[line.find("] ")+2:]
                    # check for standard message sent by a user.
                    if line[0] == "<":
                        lines.append(line)
                    # check for overhead message stating viewer count.
                    elif line[0:8] == "VIEWERS:":
                        # for now add every viewer count to a list for averaging later on.
                        data[channel]["viewers"].append(int(line[9:].replace("\xa0", "")))
                    # check for overhead message stating a user was banned
                    elif line[0:4] == "BAN:":
                        # find most recent message sent by banned user and mark as bad message.
                        for i, msg in reversed(list(enumerate(lines))):
                            if line[5:].split(" ")[0] + ">" in msg:
                                data[channel]["bad_messages"].append(len(data[channel]["messages"]) + i)
                                break
                    # check for overhead message stating a message was deleted.
                    elif line[0:8] == "DELETED:":
                        # find the deleted message and mark it as bad message.
                        for i, msg in reversed(list(enumerate(lines))):
                            if line[9:].split(" (")[0] + "> " + line[line.find(" (")+2:-1] in msg:
                                data[channel]["bad_messages"].append(i)
                                break
            # sort bad message indices.
            data[channel]["bad_messages"] = sorted(set(data[channel]["bad_messages"]))
            # remove names from messages and add to data.
            for temp in lines:
                temp = temp[temp.find(">")+2:]
                # non-functioning attempt at anonomizing @-mentions.
                #index = temp.find("@")
                #while index != -1:
                #    temp.replace(temp[index:temp.find(" ", index)], "@user")
                #    index = temp.find("@", index+1)
                data[channel]["messages"].append(temp)
# average viewer counts by channel and remove channels without viewer data.
removals = ["liqudwifi", "chrisheroes", "maudado", "ac7ionman", "shrodingerlee", "zealsambitious", "jpf_14-1"]
for channel in data.keys():
    if len(data[channel]["viewers"]) == 0:
        removals.append(channel)
    else:
        avg_viewers = int(sum(data[channel]["viewers"]) / len(data[channel]["viewers"]))
        data[channel]["viewers"] = avg_viewers
for channel in removals:
    data.pop(channel)
discarded_channels = []
for channel in data.keys():
    if data[channel]["viewers"] >= 10000 or len(data[channel]["bad_messages"]) == 0:
        discarded_channels.append(channel)

formatted_data = []
for channel in data.keys():
    if channel in discarded_channels:
        continue
    next_bad = 0
    for index in range(len(data[channel]["messages"])):
        row = []
        if next_bad < len(data[channel]["bad_messages"]) and data[channel]["bad_messages"][next_bad] == index:
            row = ["bad", data[channel]["messages"][index], channel]
            next_bad += 1
        else:
            row = ["good", data[channel]["messages"][index], channel]
        formatted_data.append(row)
df = pd.DataFrame(formatted_data, columns=["status", "message", "channel"])
# Undersampling the training data
temp = df.groupby(['status']).size()

res = []
res.append(df[df.status == "bad"])
indices = df[df.status == "good"].index
random_indices = np.random.choice(indices, temp["bad"]*9, replace=False)
res.append(df.loc[random_indices])
undersampled_data = pd.concat(res)
train, test = train_test_split(undersampled_data, test_size=0.2)

In [6]:
del(df)
del(data)
del(lines)
del(formatted_data)
del(temp)
del(res)
del(indices)
gc.collect()

0

In [7]:
train.head()

,status,message,channel
21159337,good,do it anyway\n,elajjaz
19300702,good,Happy near year guys\n,dannyaarons
24211179,good,nerf hunters? why ?\n,gingitv
21433948,good,BBoomer\n,elajjaz
11850494,good,LUL\n,39daph


from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pipe = Pipeline([('count_vectorizer', CountVectorizer()), ('tfidf', TfidfTransformer()),('bayes', MultinomialNB())])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

from sklearn.metrics import classification_report
good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

In [9]:
# Saker att testa: Support Vector Machines, XGBoost, K Nearest Neighbors, XLNet

# Naive-Bayes with undersampled training data.

pipe = Pipeline([('count_vectorizer', CountVectorizer(binary=True)), ('tfidf', TfidfTransformer()),('bayes', MultinomialNB())])
pipe.fit(undersampled_train['message'], undersampled_train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

from sklearn.model_selection import GridSearchCV
pipe = Pipeline([('count_vectorizer', CountVectorizer()), 
                 ('tfidf', TfidfTransformer()),
                 ('bayes', MultinomialNB())])
grid = GridSearchCV(pipe, {
    'count_vectorizer__binary':(True, False),
    'count_vectorizer__ngram_range':((1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3)),
    'bayes__alpha': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0),
    'bayes__fit_prior': (True, False)
})
grid.fit(train['message'], train['status'])
predict = grid.predict(test['message'])

print("Best parameter (CV score=%0.3f):" % grid.best_score_)
print(grid.best_params_)

In [8]:
# Naive Bayes with full data and selected parameters


pipe = Pipeline([
    ('count_vectorizer', CountVectorizer(
        binary=True
    )),
    ('tfidf', TfidfTransformer()),
    ('bayes', MultinomialNB(
        #alpha=grid.best_params_["bayes__alpha"],
        #fit_prior=grid.best_params_["bayes__fit_prior"]
    ))
])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

{'good': 103199, 'bad': 2229}
              precision    recall  f1-score   support

         bad       0.94      0.20      0.33     10347
        good       0.92      1.00      0.96     95081

    accuracy                           0.92    105428
   macro avg       0.93      0.60      0.65    105428
weighted avg       0.92      0.92      0.90    105428



# Naive Bayes with undersampled data and selected parameters


pipe = Pipeline([('count_vectorizer', CountVectorizer(binary=True, ngram_range=(1, 1))), ('tfidf', TfidfTransformer()),('bayes', MultinomialNB(alpha=0.4, fit_prior=False))])
pipe.fit(undersampled_train['message'], undersampled_train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

pipe = Pipeline([('count_vectorizer', CountVectorizer(binary=True)), ('tfidf', TfidfTransformer()), ('scaler', StandardScaler(with_mean=False)), ('svc', LinearSVC())])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

In [10]:
pipe = Pipeline([
    ('count_vectorizer', CountVectorizer(binary=True)),
    ('tfidf', TfidfTransformer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('svc', LinearSVC(max_iter = 50000, dual = False))
])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

{'good': 99829, 'bad': 5599}
              precision    recall  f1-score   support

         bad       0.62      0.34      0.44     10347
        good       0.93      0.98      0.95     95081

    accuracy                           0.91    105428
   macro avg       0.78      0.66      0.70    105428
weighted avg       0.90      0.91      0.90    105428



In [11]:
pipe = Pipeline([
    ('count_vectorizer', CountVectorizer(binary=True)),
    ('tfidf', TfidfTransformer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('svc', LinearSVC(max_iter = 50000, dual = False, class_weight='balanced'))
])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))

{'good': 78929, 'bad': 26499}
              precision    recall  f1-score   support

         bad       0.26      0.68      0.38     10347
        good       0.96      0.80      0.87     95081

    accuracy                           0.78    105428
   macro avg       0.61      0.74      0.62    105428
weighted avg       0.89      0.78      0.82    105428



pipe = Pipeline([('count_vectorizer', CountVectorizer()), ('tfidf', TfidfTransformer()),('svc', SVC())])
pipe.fit(undersampled_train['message'], undersampled_train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))


pipe = Pipeline([('count_vectorizer', CountVectorizer(binary=True)), 
                 ('tfidf', TfidfTransformer()),
                 ('scaler', StandardScaler(with_mean=False)),
                 ('svc', LinearSVC(dual = False))])
#grid = GridSearchCV(pipe, {
#    'count_vectorizer__binary':(True, False),
#    'count_vectorizer__ngram_range': ((1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3)),
#    'svc__C': (0.1, 0.5, 1.0, 5.0, 10.0),
#    'svc__kernel': ("linear", "poly", "rbf", "sigmoid", "precomputed"),
#    'svc__degree': (1, 2, 3, 4, 5, 10),
#    'svc__gamma': ("auto", "scale"),
#    'svc__shrinking': (True, False),
#    'svc__probability': (True, False),
#    'svc__tol': (0.001, 0.005, 0.01, 0.05, 0.1),
#    'svc__class_weight': (None, "balanced")})

grid = GridSearchCV(pipe, {
    'svc__C': (0.8, 0.9, 1.0, 1.1),
    #'svc__kernel': ("linear", "poly", "rbf", "sigmoid", "precomputed"),
    #'svc__degree': (2, 3, 4),
    #'svc__gamma': ("auto", "scale"),
    #'svc__shrinking': (True, False),
    #'svc__probability': (True, False),
    'svc__class_weight': (None, "balanced"),
    #'svc__tol': (0.009, 0.01, 0.011)
})
grid.fit(train['message'], train['status'])
predict = grid.predict(test['message'])

print("Best parameter (CV score=%0.3f):" % grid.best_score_)
print(grid.best_params_)

pipe = Pipeline([
    ('count_vectorizer', CountVectorizer(
        binary=True
    )),
    ('tfidf', TfidfTransformer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('svc', LinearSVC(
        C=grid.best_params_["svc__C"],
        #kernel=grid.best_params_["svc__kernel"],
        #degree=grid.best_params_["svc__degree"],
        #gamma=grid.best_params_["svc__gamma"],
        #shrinking=grid.best_params_["svc__shrinking"],
        #probability=grid.best_params_["svc__probability"],
        class_weight=grid.best_params_["svc__class_weight"],
        #tol=grid.best_params_["svc__tol"],
        dual=False,
        max_iter=10000
    ))
])
pipe.fit(train['message'], train['status'])

predict = pipe.predict(test['message'])

good_bad_count = {"good": 0, "bad": 0}
for guess in predict:
        good_bad_count[guess] += 1
print(good_bad_count)
print(classification_report(test['status'], predict))